In [152]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from arch import arch_model
import plotly.express as px
from python_module.pricing_model import SABRModel, BlackScholesModel

In [153]:
# -------
# PARAMETERS
# -------
F = 100
T = 250/250
alpha = 0.1
beta = 1
rho = -0.9
nu = 0.5
n_steps = 250
n_paths = 10
r = 0

In [154]:
# -------
# SIMULATE MONTECARLO PATH USING SABR MODEL
# -------
spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=True)
spot = spot[4]
vol = vol[4]

In [155]:
# -------
# FIT EGARCH MODEL
# -------
returns = 100 * spot.pct_change().dropna()
model = arch_model(returns, vol='EGARCH', p=1, q=1, o=1, dist='t', mean='Zero')
res = model.fit(disp='off')


/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:766: ConvergenceWarning:

The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.




In [156]:
# Set simulation parameters
n_simulations = 10000  # number of simulation paths
horizon = 250         # simulation horizon (e.g., trading days)

# Container for simulated return paths
simulated_returns = np.empty((n_simulations, horizon))

# Run simulations using the model's simulate method
for i in range(n_simulations):
    sim = model.simulate(res.params, horizon)
    simulated_returns[i, :] = sim['data']

df = pd.DataFrame(simulated_returns).transpose()/100
new_row = pd.DataFrame(index=[0], columns=df.columns)
new_row.loc[0, :] = 0
df = pd.concat([new_row, df], ignore_index=True)
ST = (df.add(1).cumprod()*100).iloc[-1]

In [157]:
px.histogram(ST)

In [158]:
sabr_skew = SABRModel.compute_sigma(F=F, K=95, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu)-SABRModel.compute_sigma(F=F, K=105, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu)

In [159]:
put_price = (95 - ST).clip(0).mean()
call_price = (ST - 105).clip(0).mean()
atm_price = (ST - 100).clip(0).mean()

In [160]:
put_vol = BlackScholesModel.solve_sigma(S=F, K=95, T=1, r=0.0, market_price=put_price, option_type='put', sigma_init=0.1, tol=1e-5, max_iter=1000)
call_vol = BlackScholesModel.solve_sigma(S=F, K=105, T=1, r=0.0, market_price=call_price, option_type='call', sigma_init=0.1, tol=1e-5, max_iter=1000)
atm_vol = BlackScholesModel.solve_sigma(S=F, K=100, T=1, r=0.0, market_price=atm_price, option_type='call', sigma_init=0.1, tol=1e-5, max_iter=1000)

In [161]:
put_vol-call_vol

0.03478366081604484

In [162]:
sabr_skew

0.02212360370027021

In [163]:
atm_vol-SABRModel.compute_sigma(F=F, K=100, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu)

0.13294787844105688